In [0]:
!pip install music21

# Importing files from google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Importing the required libraries

In [0]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf



# Function for training the network


In [0]:
def train_network():
  #Getting the notes of the music
  
  notes = get_notes()
  
  #defining the length of our vocabulary
  
  n_vocab = len(set(notes))
  
  #preparing sequences
  
  network_input, network_output = prepare_sequences(notes, n_vocab)
  
  #Creating the model
  
  model=create_network(network_input, n_vocab)
  
  #training the model
  
  train(model, network_input, network_output)

# Getting the notes from midi files

In [0]:
def get_notes():
  
  notes = []
  
  for file in glob.glob("drive/My Drive/RapMusic/*.mid"):
    
    midi = converter.parse(file)
    
    print("Parsing %s" %file)
    
    parts = instrument.partitionByInstrument(midi)
    
    notes_to_parse = None
    
    # file has instrument parts
    
    if parts: 
      
      notes_to_parse = parts.parts[0].recurse
    
    else: 
    # file has notes in a flat structure
      
      notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    
    if isinstance(element, note.Note):
      
      notes.append(str(element.pitch))
      
    elif isinstance(element, chord.Chord):
      
      notes.append('.'.join(str(n) for n in element.normalOrder))
            

            
  return notes


     

# Preparing sequences for your model

In [0]:
def prepare_sequences(notes, n_vocab):
  
  sequence_length = 100
  
  pitchnames=sorted(set(item for item in notes))
  
  note_to_int=dict((note,number) for number,note in enumerate(pitchnames))
  
  network_input = []
  
  network_output = []
  
  for i in range(0,len(notes)-sequence_length,1):
    
    sequence_in = notes[i:i+sequence_length]
    
    sequence_out = notes[i + sequence_length]
    
    network_input.append([note_to_int[char] for char in sequence_in])
    
    network_output.append(note_to_int[sequence_out])
    
  n_patterns=len(network_input)
  
  network_input = np.reshape(network_input, (n_patterns,sequence_length,1))
  
  network_input = network_input/float(n_vocab)
  
  network_output=np_utils.to_categorical(network_output)
    
  return (network_input, network_output)
    

# Creating the model

In [0]:
def create_network(network_input, n_vocab):
  
  model=Sequential()
  
  model.add(LSTM(512,input_shape=(network_input.shape[1],network_input.shape[2]),return_sequences=True))
  
  model.add(Dropout(0.4))
  
  model.add(LSTM(512,return_sequences=True))
  
  model.add(Dropout(0.3))
  
  model.add(LSTM(512))
  
  model.add(Dense(256))
  
  model.add(Dropout(0.3))
  
  model.add(Dense(n_vocab))
  
  model.add(Activation('softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
  
  return model


# Training your model

In [0]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()